# Preparación y curación de los datos

---
## Convenciones

### Dataframes principales

`msg_df` dataframe de mensajes

`md_df` dataframe de metadata

`agg_msg_df` dataframe de mensajes totalizado

`res_df` dataframe resultante del merge del dataframe de metadata y el dataframe de mensajes totalizado

### Prefijos de columnas nuevas

`n_` cantidad count

`n_words` cantidad de palabras

`avg_` promedio

---


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from ast import literal_eval
from collections import Counter

import re

from columns_to_keep import C2K
from type_to_fix import T2F

In [2]:
data_dir = os.path.join('..', 'dataset')

# dataset de mensajes
msg_data_file_name = 'dev_yup_messages_preprocessed.csv'
#msg_data_file_name = 'train_yup_messages_preprocessed.csv'

msg_full_data_file_name = os.path.join(data_dir, msg_data_file_name)
msg_df = pd.read_csv(msg_full_data_file_name)
msg_df = msg_df.astype({'created_at': 'datetime64[ns, UTC]'})

# dataset de metadata
md_data_file_name = 'datadump-20150801-20171219.csv'
md_full_data_file_name = os.path.join(data_dir, md_data_file_name)
md_df = pd.read_csv(md_full_data_file_name, low_memory=False)

# dataset de metadata
md_df.columns = [column.strip() for column in md_df.columns]
md_df = md_df[C2K]
md_df = md_df.astype(T2F)

# Caracterización del dataset de mensajes
print ('\nDimensiones del dataset de mensajes')
print ('Filas: {}'.format(msg_df.shape[0]))
print ('Columnas: {}'.format(msg_df.shape[1]))
print ('Cantidad de sesiones: ', msg_df['session_id'].nunique())
print ('Cantidad de turnos: {}'.format(msg_df.shape[0]))
print ('Cantidad de turnos del tutor: {}'.format(len(msg_df[msg_df.sent_from=='tutor'])))
print ('Cantidad de turnos del estudiante: {}'.format(len(msg_df[msg_df.sent_from=='student'])))

# Caracterización del dataset de metadata
print ('\nDimensiones del dataset de metadata')
print ('Filas: {}'.format(md_df.shape[0]))
print ('Columnas: {}'.format(md_df.shape[1]))
print ('Cantidad de sesiones: ', md_df['session_id'].nunique())


Dimensiones del dataset de mensajes
Filas: 234375
Columnas: 6
Cantidad de sesiones:  3851
Cantidad de turnos: 234375
Cantidad de turnos del tutor: 120536
Cantidad de turnos del estudiante: 89706

Dimensiones del dataset de metadata
Filas: 63265
Columnas: 39
Cantidad de sesiones:  63265


In [3]:
def student_rating_category(x):
    
    if x <= 2:
        return 'negativa'
    
    if x >= 4:
        return 'positiva'

    return 'neutra'

In [4]:
# Cálculo de las longitudes de los mensajes (cantidad de palabas)

msg_df['tokens'] = msg_df.text.apply(lambda x: literal_eval(x))
msg_df['n_words'] = msg_df.tokens.apply(lambda x: len(x))

In [5]:
# Número de palabras tutor y estudiante

agg_tutor_msg_df = msg_df[msg_df['sent_from']=='tutor'].groupby('session_id').agg(
    avg_words_tutor = ('n_words', np.mean),
    n_words_tutor = ('n_words', sum),
    n_msg_tutor = ('sent_from', lambda x: x.eq('tutor').sum())
    
)
agg_student_msg_df = msg_df[msg_df['sent_from']=='student'].groupby('session_id').agg(
    avg_words_student = ('n_words', np.mean),
    n_words_student = ('n_words', sum),
    n_msg_student = ('sent_from', lambda x: x.eq('student').sum())
    
)

In [6]:
# Otras agregaciones del dataset de mensajes 
agg_msg_df = msg_df.groupby('session_id').agg(
        # started =('created_at', min),
        # ended =('created_at', max),
        duration = ('created_at', lambda x: (max(x) - min(x))),
        n_msg_bot = ('sent_from', lambda x: x.eq('bot').sum()),
        n_msg_system = ('sent_from', lambda x: x.isin(list(['system alert', 'system info', 'system warn'])).sum()),
        n_msg_content_text = ('content_type', lambda x: x.eq('text').sum()),
        n_msg_content_image = ('content_type', lambda x: x.eq('image').sum()),
)

In [7]:
print(agg_tutor_msg_df.shape)
print(agg_student_msg_df.shape)
print(agg_msg_df.shape)

(3338, 3)
(3767, 3)
(3851, 5)


In [8]:
# Joins
res_df = pd.merge(md_df, agg_msg_df, on=["session_id", "session_id"])
res_df = pd.merge(res_df, agg_tutor_msg_df, how="left", on=["session_id", "session_id"])
res_df = pd.merge(res_df, agg_student_msg_df, how="left", on=["session_id", "session_id"])

# Llenar los NAN producto del left join con ceros
res_df['avg_words_tutor'] = res_df['avg_words_tutor'].fillna(0)
res_df['n_words_tutor'] = res_df['n_words_tutor'].fillna(0)
res_df['n_msg_tutor'] = res_df['n_msg_tutor'].fillna(0)

res_df['avg_words_student'] = res_df['avg_words_student'].fillna(0)
res_df['n_words_student'] = res_df['n_words_student'].fillna(0)
res_df['n_msg_student'] = res_df['n_msg_student'].fillna(0)

In [9]:
res_df = res_df.dropna(subset=['student_rating'])
res_df['student_rating_cat'] = res_df.student_rating.apply(student_rating_category)

In [12]:
# Caracterización del dataset combinado
print ('\nDimensiones del dataset combinado')
print ('Filas: {}'.format(res_df.shape[0]))
print ('Columnas: {}'.format(res_df.shape[1]))
print ('Cantidad de sesiones: ', res_df['session_id'].nunique())


Dimensiones del dataset combinado
Filas: 2745
Columnas: 51
Cantidad de sesiones:  2745


In [11]:
# Salva archivo csv con el prefijo del archivo procesado
curated_data_file_name = msg_data_file_name.split('_')[0] + '_curated_data.csv'
curated_full_data_file_name = os.path.join(data_dir, curated_data_file_name)
res_df.to_csv(curated_full_data_file_name)